In [32]:
# Importamos utilidades para manejar rutas de forma segura (independiente del SO)
from pathlib import Path
# Importamos módulos del sistema y pandas para leer el CSV
import os, pandas as pd

# Ruta ABSOLUTA a la carpeta base del proyecto en WSL
BASE = Path("/home/luisrueda/PROYECTO-IA")
# Construimos la ruta al CSV uniendo carpetas/archivo de forma segura
RUTA = BASE / "data" / "IoT_AquaSensors_Crudo.csv"

# Mostramos el directorio actual donde corre Python (útil para depurar rutas)
print("CWD Python:", os.getcwd())
# Mostramos la ruta completa del archivo que intentaremos leer
print("Archivo que voy a leer:", RUTA)

# Verificamos que el archivo exista; si no, cortamos con un error claro
assert RUTA.exists(), f"No se encontró: {RUTA}"

# Leemos el CSV como texto (dtype=str) para tener control total de tipos; 
# low_memory=False evita inferencias parciales de tipos en archivos grandes
df = pd.read_csv(RUTA, dtype=str, low_memory=False)
# Confirmamos dimensiones del DataFrame (filas, columnas)
print("OK, cargado:", df.shape)
# Imprimimos las primeras 12 columnas para inspección rápida
print("Columnas:", list(df.columns)[:12])

CWD Python: /home/luisrueda/PROYECTO-IA/salidas
Archivo que voy a leer: /home/luisrueda/PROYECTO-IA/data/IoT_AquaSensors_Crudo.csv
OK, cargado: (74796, 10)
Columnas: ['station', 'Date', 'Time', 'NITRATE(PPM)', 'PH', 'AMMONIA(mg/l)', 'TEMP', 'DO', 'TURBIDITY', 'MANGANESE(mg/l)']


In [40]:
# Paso 1. Normalización y renombrado de variables
#Se realizó una estandarización inicial de la estructura del conjunto de datos para garantizar coherencia semántica 
# y facilitar su tratamiento posterior. Primero, se homogenizaron los nombres de las columnas convirtiéndolos a 
# minúsculas y eliminando espacios en blanco; esto evita ambigüedades y fallos al referenciar variables en el
# código. Luego, se aplicó un mapeo a nombres canónicos en español, alineados con la terminología del dominio 
# acuícola y los objetivos del estudio. En particular: station→estacion, date→fecha, time→hora, temp→temperatura_c,
# do→oxigeno_disuelto_mg_l, turbidity→turbidez_ntu, ammonia(mg/l)→amoniaco_mg_l, nitrate(ppm)→nitrato_ppm, 
# manganese(mg/l)→manganeso_mg_l. Este paso mejora la legibilidad, reduce errores en las etapas de limpieza/transformación 
# y deja un esquema de variables consistente para reproducibilidad y trazabilidad del análisis.

df = df.copy()# Hago una copia para no modificar el df original por accidente
df.columns = [c.strip().lower() for c in df.columns] # ← quita espacios y pasa nombres de columnas a minúsculas
# Mapa típico de las columnas
renombrar_es = {
    "station":        "estacion",
    "date":           "fecha",
    "time":           "hora",
    "nitrate(ppm)":   "nitrato_ppm",
    "ph":             "ph",
    "ammonia(mg/l)":  "amoniaco_mg_l",
    "temp":           "temperatura_c",
    "do":             "oxigeno_disuelto_mg_l",
    "turbidity":      "turbidez_ntu",
    "manganese(mg/l)":"manganeso_mg_l",
}
df = df.rename(columns=renombrar_es)  # ← aplica el diccionario: solo cambia las columnas que existan##
print(df.head(20).to_string(index=False))# Ver las primeras 20 filas como tabla en consola

estacion      fecha     hora nitrato_ppm  ph amoniaco_mg_l temperatura_c oxigeno_disuelto_mg_l turbidez_ntu manganeso_mg_l
station1 01-02-2022 08:00:00        18.3 5.7          0.01          23.2                  11.6         31.7           0.71
station1 01-02-2022 08:20:00         3.6 5.1         0.094         23.41                  10.5         18.8           0.62
station1 01-02-2022 08:40:00        13.1 5.5          0.06         23.63                  10.3         23.2           0.73
station1 01-02-2022 09:00:00        18.1 5.2         0.018         23.64                   9.4         26.7           0.64
station1 01-02-2022 09:20:00        10.8 5.2         0.038         23.81                   8.8         19.5           0.68
station1 01-02-2022 09:40:00         1.8   5         0.039         23.85                   7.3         18.8           0.67
station1 01-02-2022 10:00:00          22 5.3         0.057          24.1                  11.5           32           0.63
station1 01-02-2

In [43]:
# PASO 2 — Convertir a numérico (coma → punto)
#Define qué columnas deben ser numéricas (parámetros de agua)
#Recorre solo las que realmente existan en el DataFrame.
#Estandariza decimales: cambia coma por punto (ej. 7,2 → 7.2).
#Convierte cada columna a tipo numérico con pd.to_numeric(...).
#Controla errores: si hay texto u otros valores inválidos, los convierte a NaN (no revienta el proceso).
#Resultado: esas variables quedan en formato float/int, listas para cálculos, limpieza por rangos e imputación.

import numpy as np                     # ← usamos NumPy por si luego necesitamos operaciones numéricas/NaN

# ← lista de columnas que QUEREMOS que sean numéricas
num_cols = ["nitrato_ppm","ph","amoniaco_mg_l","temperatura_c",
            "oxigeno_disuelto_mg_l","turbidez_ntu","manganeso_mg_l"]

# ← recorremos cada nombre de columna en la lista
for col in num_cols:
    if col in df.columns:              # ← solo actuamos si la columna existe en el DataFrame
        df[col] = pd.to_numeric(       # ← convierte a tipo numérico (float / int)
            df[col]                   #    toma la columna
              .astype(str)            #    la convierte a string (por si venía mezclada)
              .str.replace(",", ".",  #    cambia coma decimal por punto (p. ej., "7,2" → "7.2")
                            regex=False),
            errors="coerce"            # ← si hay texto inválido, lo convierte a NaN en vez de dar error
        )

# ← pequeña inspección: muestra el tipo final de cada columna convertida
{c: str(df[c].dtype) for c in num_cols if c in df.columns}


{'nitrato_ppm': 'float64',
 'ph': 'float64',
 'amoniaco_mg_l': 'float64',
 'temperatura_c': 'float64',
 'oxigeno_disuelto_mg_l': 'float64',
 'turbidez_ntu': 'float64',
 'manganeso_mg_l': 'float64'}

In [44]:
# PASO 3 — marca_tiempo (fecha+hora) y orden
#Construye una marca temporal única (marca_tiempo) combinando fecha y hora.
#Parsea ese texto a tipo datetime; si no puede, lo deja como NaT (valor de fecha/hora faltante).
#Usa dayfirst=True, asumiendo formato DD/MM/AAAA (útil en datos locales).
#Si faltara la columna estacion, la crea como "desconocida" para poder agrupar.
#Ordena el dataset por estacion y marca_tiempo, dejando la tabla lista para análisis temporal 
# (lags, ventanas móviles, detección de caídas bruscas, etc.).
def construir_ts(r):
    f = str(r.get("fecha", "") or "")   # ← toma 'fecha'; si no existe o viene None, usa ""
    h = str(r.get("hora", "")  or "")   # ← toma 'hora'; si no existe o viene None, usa ""
    # ← si hay 'hora' válida, concatena "fecha hora"; si no, devuelve solo 'fecha'
    return f"{f} {h}" if h and h.lower() not in ("nan", "none") else f

# ← crea columna 'marca_tiempo' convirtiendo texto a datetime
#    errors='coerce' pone NaT cuando no puede parsear
#    dayfirst=True interpreta DD/MM/YYYY como día-primero (ajústalo si usas MM/DD/YYYY)
df["marca_tiempo"] = pd.to_datetime(
    df.apply(construir_ts, axis=1),
    errors="coerce",
    infer_datetime_format=True,
    dayfirst=True
)

# ← si no existe columna 'estacion', créala para poder agrupar/ordenar
if "estacion" not in df.columns:
    df["estacion"] = "desconocida"

# ← ordena cronológicamente por estación y marca de tiempo; reinicia índice limpio
df = df.sort_values(["estacion", "marca_tiempo"]).reset_index(drop=True)

# ← vista rápida para verificar
df[["estacion", "marca_tiempo"]].head()

/tmp/ipykernel_46056/457467362.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["marca_tiempo"] = pd.to_datetime(


,estacion,marca_tiempo
0,Station2,2022-02-01 00:00:00
1,Station2,2022-02-01 00:20:00
2,Station2,2022-02-01 00:40:00
3,Station2,2022-02-01 01:00:00
4,Station2,2022-02-01 01:20:00


In [ ]:
# PASO 4 — depura valores imposibles
#Define rangos plausibles para cada parámetro (pH, temperatura, DO, turbidez, amoníaco, nitrato, manganeso) 
# basados en límites físico-químicos razonables para acuicultura.
#Revisa cada columna y crea una máscara que detecta lecturas no nulas fuera de esos rangos 
# (posibles errores de sensor, unidades mal registradas o outliers imposibles).
#Sustituye esas lecturas fuera de rango por NaN para no sesgar análisis/modelos; estos NaN se imputarán en el Paso 5.
#Registra cuántos valores fueron reemplazados por variable en el dict reemplazos, lo que te da un resumen de calidad y trazabilidad de la limpieza.
# Diccionario: para cada variable, (mínimo_aceptable, máximo_aceptable)
rangos = {
    "ph":                     (0, 14),    # pH físico-químico típico
    "temperatura_c":         (0, 50),    # °C plausibles en acuicultura
    "oxigeno_disuelto_mg_l": (0, 20),    # mg/L
    "turbidez_ntu":          (0, 1000),  # NTU (muy amplio)
    "amoniaco_mg_l":         (0, 10),    # mg/L
    "nitrato_ppm":           (0, 200),   # ppm
    "manganeso_mg_l":        (0, 10),    # mg/L
}

reemplazos = {}  # ← aquí guardaremos cuántos valores fueron marcados como NaN por variable

# Recorremos cada variable y su par (mín, máx)
for col, (lo, hi) in rangos.items():
    if col in df.columns:  # ← solo si la columna existe en el DataFrame
        # Máscara: valores no nulos y que están fuera del rango permitido
        mask = df[col].notna() & ((df[col] < lo) | (df[col] > hi))
        # Guardamos cuántos se van a reemplazar (diagnóstico)
        reemplazos[col] = int(mask.sum())
        # Ponemos NaN en los fuera de rango (se imputarán en el PASO 5)
        df.loc[mask, col] = np.nan

# Mostrar resumen: cuántos valores se marcaron como NaN por estar fuera de rango
reemplazos

{'ph': 0,
 'temperatura_c': 0,
 'oxigeno_disuelto_mg_l': 0,
 'turbidez_ntu': 0,
 'amoniaco_mg_l': 0,
 'nitrato_ppm': 0,
 'manganeso_mg_l': 0}

In [48]:
# ===== PASO 5: Imputación de faltantes por estación (mediana; respaldo global) =====
#Objetivo: rellenar los valores faltantes (NaN) en las variables numéricas respetando el contexto de cada estación.
#Cómo lo hace: para cada columna numérica, calcula la mediana por estacion y reemplaza los NaN de esa estación con su mediana.
#Si una estación no tiene datos suficientes para calcular mediana, usa la mediana global de la columna como respaldo.
#Diagnóstico: imprime el porcentaje de NaN antes y después para que verifiques el impacto de la imputación.
#Salida: produce df_limpio (y luego lo asigna a df) con los huecos cubiertos, dejando el dataset coherente y completo para los pasos siguientes.
#Notas: si alguna columna queda con NaN es porque toda la columna estaba vacía (no hay información para imputar).
import numpy as np  # ← para manejar NaN con seguridad

# Lista de columnas numéricas a imputar (se filtran solo las que existan en el df)
cols_num = [c for c in [
    "nitrato_ppm","ph","amoniaco_mg_l","temperatura_c",
    "oxigeno_disuelto_mg_l","turbidez_ntu","manganeso_mg_l"
] if c in df.columns]

# (Opcional) Diagnóstico antes de imputar: porcentaje de NaN por columna
print("NaN antes (%):", (df[cols_num].isna().mean().round(3)*100).astype(str).to_dict())

def imputar_por_estacion(dataframe, columnas):
    """Rellena NaN usando la mediana por 'estacion'; si no hay datos en esa estación, usa la mediana global."""
    df_tmp = dataframe.copy()  # ← trabajamos sobre una copia para no alterar el original
    for col in columnas:       # ← recorremos cada columna numérica a imputar
        med_global = df_tmp[col].median(skipna=True)  # ← mediana global de respaldo para esta columna
        # Aplicamos por estación: calcula la mediana de la serie en esa estación y rellena sus NaN
        df_tmp[col] = df_tmp.groupby("estacion")[col].transform(
            lambda s: s.fillna(s.median(skipna=True) if not np.isnan(s.median(skipna=True)) else med_global)
        )
    return df_tmp  # ← devolvemos el dataframe con NaN imputados

# Ejecutamos la imputación sobre las columnas numéricas seleccionadas
df_limpio = imputar_por_estacion(df, cols_num)

# (Opcional) Diagnóstico después de imputar: debería tender a 0% si había datos suficientes
print("NaN después (%):", (df_limpio[cols_num].isna().mean().round(3)*100).astype(str).to_dict())

# Conteo total de NaN restantes en columnas numéricas (idealmente 0)
total_nulos = int(df_limpio[cols_num].isna().sum().sum())
print(f"PASO 5 OK → nulos restantes en numéricas: {total_nulos}")

# Importante: seguir trabajando desde df_limpio a partir de aquí
df = df_limpio  # ← actualizamos 'df' para los siguientes pasos

NaN antes (%): {'nitrato_ppm': '0.0', 'ph': '0.0', 'amoniaco_mg_l': '0.0', 'temperatura_c': '0.0', 'oxigeno_disuelto_mg_l': '0.0', 'turbidez_ntu': '0.0', 'manganeso_mg_l': '0.0'}
NaN después (%): {'nitrato_ppm': '0.0', 'ph': '0.0', 'amoniaco_mg_l': '0.0', 'temperatura_c': '0.0', 'oxigeno_disuelto_mg_l': '0.0', 'turbidez_ntu': '0.0', 'manganeso_mg_l': '0.0'}
PASO 5 OK → nulos restantes en numéricas: 0


In [ ]:
# ===== PASO 6: Transformación con escalado robusto (añade columnas *_z) =====
#Objetivo: normalizar la escala de las variables numéricas para que sean comparables y robustas a outliers.
#Cómo lo hace: aplica RobustScaler (centra por mediana y escala por IQR). 
#No reemplaza tus columnas originales: agrega nuevas con sufijo _z (ej.: ph_z, temperatura_c_z).
#Entrada: el df ya imputado en el Paso 5 (sin NaN en numéricas).
#Salida: un df_transformado (que luego reasignas a df) con todas las columnas originales + las versiones escaladas _z.
#Para qué sirve: muchos modelos (especialmente de detección de anomalías y distancia) funcionan mejor cuando las
# features están en la misma escala y sin sesgo por valores extremos.

from sklearn.preprocessing import RobustScaler   # ← importamos el escalador robusto (menos sensible a outliers)

# ← columnas numéricas que queremos escalar (ajusta si falta alguna en tu df)
cols_num = [c for c in [
    "nitrato_ppm","ph","amoniaco_mg_l","temperatura_c",
    "oxigeno_disuelto_mg_l","turbidez_ntu","manganeso_mg_l"
] if c in df.columns]

escalador = RobustScaler()                       # ← creamos el objeto escalador
Xz = escalador.fit_transform(df[cols_num])       # ← ajusta con tus datos y transforma → matriz numpy escalada

df_transformado = df.copy()                      # ← trabajamos sobre una copia para conservar las columnas originales
for i, c in enumerate(cols_num):                 # ← recorremos cada columna numérica en el mismo orden
    df_transformado[c + "_z"] = Xz[:, i]         # ← añadimos la versión escalada con sufijo "_z"

print("PASO 6 OK → columnas escaladas añadidas:", [c + "_z" for c in cols_num][:8])  # ← muestra algunas nuevas
print("Forma final:", df_transformado.shape)     # ← filas y columnas después de agregar las escaladas

# (Opcional) a partir de aquí seguimos usando df_transformado como dataset transformado
df = df_transformado                              # ← actualizamos df para pasos posteriores (si los hubiera)

PASO 6 OK → columnas escaladas añadidas: ['ph_z']
Forma final: (74796, 12)


/home/luisrueda/PROYECTO-IA/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1216: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a, func=_nanmedian, keepdims=keepdims,
/home/luisrueda/PROYECTO-IA/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1406: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
